In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def read_json(f):
    f = open(f) 
    return json.load(f) 



def gen_plots(path_dir = "/home/*****/work/pid/results/sim_grad/seed_{}/setting_{}/dim_{}/rho0.7.json"
              ,sigma=0.7,seeds=[],setting=0,dim=10,mets=[]):
    gt = {}
    e= {met : []  for met in mets}
    e_s= {met : []   for met in mets}
    sigma = str(sigma)
    for i, seed in enumerate( seeds ) :
            file_path = path_dir.format(seed,setting,dim)
            print("doing_file "+ str(file_path))
            out=read_json(file_path)
            for idx, met in enumerate(mets):
                gt[met]= out[sigma] ["gt"]["g_o_inf"][idx]
                e[met].append(out[sigma] ["e"]["g_o_inf"][met]["simple"])
                e_s[met].append(out[sigma] ["e"]["g_o_inf"][met]["sigma"])
                
    return {"gt":gt, "e": e,"e_s":e_s}


In [ ]:

for setting in [0,1]:
    for dim in [5,10,15]:
        if setting ==0:
            NB = 6
        else :
            NB = 10
        data = gen_plots(seeds=[22,11,55,42,33],
                         mets=["x"+str(i) for i in range(NB)],
                         setting=setting,
                        dim=dim)

        
        

        mods =  data["e"].keys()

   
        
        data_plot = { mod : data["e"][mod] for mod in mods }
        df = pd.DataFrame(data_plot)
        df = df.melt(value_vars=mods,var_name=["var"],value_name="val")

        #labels = [r'$X^{i}$'.format(i) for i in range(10)]
        mylabels = [ r'$X^{1}$',r'$X^{2}$',r'$X^{3}$',r'$X^{4}$',r'$X^{5}$',r'$X^{6}$',r'$X^{7}$',r'$X^{8}$',r'$X^{9}$',r'$X^{10}$']
        labels = mylabels[:NB]


        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(4, 3))


        my_pal={"x"+str(i): "steelblue" if  data["gt"]["x"+str(i)]>0 else "tomato" for i in range(NB)}
        bplot1  = sns.boxplot(x='var', y='val', data=df, ax=ax1, palette=my_pal,fliersize=0)
        ax1.set_xticklabels(labels)

        plt.scatter(np.arange(NB),[data["gt"]["x"+str(i)] for i in range(NB)], 
                    zorder=2,marker='D',s=35,color = 'dimgray',
                    label="GT")
        
        plt.plot(np.arange(NB),0*np.arange(NB), 
                    zorder=-1,color="black",linestyle=":")
            

                

        for ax in [ax1]:
            #ax.yaxis.grid(True)
            ax.set_xlabel('',)
            ax.set_ylabel(r'$\partial \Omega $',fontsize=14)
            plt.yticks(fontsize=12)
            plt.xticks(fontsize=12)
            ax.legend(fontsize=13)
        #plt.show()
        plt.savefig("plots_g/grad_setting_{}_dim_{}.png".format(setting,dim),bbox_inches='tight',dpi=300)